In [78]:
# Basic Packages
import pandas as pd
import numpy as np
import os

# NLP Packages
import nltk 
from nltk.corpus import stopwords

# WordCloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Sklearn Packages
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, plot_confusion_matrix, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import ConvergenceWarning
from imblearn.over_sampling import SMOTE

# Pandas Settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# Solve warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Import pickle
import pickle

In [21]:
df = pd.read_csv('../csv/Hotel_Review_Spell_Checked.csv', index_col=0)

# Preprocessing

In [22]:
stop_words = stopwords.words('english')

In [23]:
df.isna().sum()

Unnamed: 0.1       0
Hotel_Name         0
Negative_Review    0
Positive_Review    0
Reviewer_Score     0
Reviews_Clean      0
Score              0
Spell_Checked      0
dtype: int64

In [24]:
df.dropna(inplace=True)

## Count Vectorizer

In [25]:
# Instantiate CountVectorizer
cv = CountVectorizer(stop_words=stop_words)

# Fit and transform dataframe without data cleaning
df_cv = cv.fit_transform(df['Spell_Checked'])
df_cv = pd.DataFrame(df_cv.toarray(), columns = cv.get_feature_names())
df_cv.index = df.index

In [26]:
# # Using Sparse in the DataFrame
# df_sparse = df_cv.astype('Sparse')

In [27]:
y = df.Score
X_cv = df_cv

In [28]:
# Running Train Test Split
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_cv, y, test_size= 0.25)

## TF-IDF

In [29]:
def fn_tdm_tfidf(docs, xColNames = None, **kwargs):

    #initialize the  vectorizer
    tf = TfidfVectorizer(**kwargs)
    x1 = tf.fit_transform(docs)
    #create dataFrame
    df = pd.DataFrame(x1.toarray().transpose(), index = tf.get_feature_names())

    if xColNames is not None:
        df.columns = xColNames

    return df

In [30]:
df_tfidf = fn_tdm_tfidf(df['Spell_Checked']).transpose()

In [31]:
df_tfidf.drop(columns='zafirovski', inplace=True)

In [32]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(df_tfidf, y, test_size=0.25, random_state=1)

## Evaluation Metric

In [33]:
# Evaluation function

def evaluation(y_true, y_pred):
    
# Print Accuracy, Recall, F1 Score, and Precision metrics.
    print('Evaluation Metrics:')
    print('Precision: ' + str(metrics.precision_score(y_true, y_pred)))
    print('Accuracy: ' + str(metrics.accuracy_score(y_true, y_pred)))
    print('F1 Score Weighted: ' + str(metrics.f1_score(y_true, y_pred, average="weighted")))
    print('F1 Score Macro: ' + str(metrics.f1_score(y_true, y_pred, average="macro")))
    print('Recall: ' + str(metrics.recall_score(y_true, y_pred)))

## Modeling With Count Vectorizer

### Logistic Regression

In [57]:
# Baseline Regression Model
lg_base = LogisticRegression()
lg_base.fit(X_train_cv, y_train_cv) 
y_lg_base_cv = lg_base.predict(X_test_cv)

# 28 seconds

In [58]:
# Logistic Regression baseline evaluation
evaluation(y_test_cv, y_lg_base_cv)

Evaluation Metrics:
Precision: 0.8269654665686995
Accuracy: 0.8142460684551341
F1 Score Weighted: 0.8142858985513343
F1 Score Macro: 0.8141811839435822
Recall: 0.8085488505747126


### Random Forest

In [59]:
rf_cv = RandomForestClassifier(n_jobs=-1)
rf_cv.fit(X_train_cv, y_train_cv)
y_rf_cv = rf_cv.predict(X_test_cv)

# 50 seconds

In [60]:
# Random Forest baseline evaluation
evaluation(y_test_cv, y_rf_cv)

Evaluation Metrics:
Precision: 0.8384615384615385
Accuracy: 0.8105457909343201
F1 Score Weighted: 0.8105207323497989
F1 Score Macro: 0.8105429309871737
Recall: 0.7830459770114943


### Naive Bayes

In [61]:
nb_base_cv = GaussianNB()
nb_base_cv.fit(X_train_cv, y_train_cv)

y_nb_base_cv = nb_base_cv.predict(X_test_cv)

In [62]:
# Naive Bayes baseline evaluation
evaluation(y_test_cv, y_nb_base_cv)

Evaluation Metrics:
Precision: 0.5603777827749045
Accuracy: 0.5842738205365402
F1 Score Weighted: 0.5354939844510527
F1 Score Macro: 0.5307127978714168
Recall: 0.8951149425287356


## Modeling with TF-IDF

### Logistic Regression

In [63]:
# Baseline Regression Model
lg_base_tfidf = LogisticRegression()
lg_base_tfidf.fit(X_train_tfidf, y_train_tfidf) 
y_lg_base_tfidf = lg_base_tfidf.predict(X_test_tfidf)
# 28 seconds

In [64]:
# Logistic Regression TF-IDF baseline evaluation
evaluation(y_test_tfidf, y_lg_base_tfidf)

Evaluation Metrics:
Precision: 0.8516104294478528
Accuracy: 0.8210915818686402
F1 Score Weighted: 0.8210963096449203
F1 Score Macro: 0.8210914838836396
Recall: 0.7929310960371296


### Random Forest 

In [65]:
rf_base_tfidf = RandomForestClassifier()
rf_base_tfidf.fit(X_train_tfidf, y_train_tfidf)
y_rf_base_tfidf = rf_base_tfidf.predict(X_test_tfidf)

# 50 seconds

In [66]:
# Logistic Regression TF-IDF baseline evaluation
evaluation(y_test_tfidf, y_rf_base_tfidf)

Evaluation Metrics:
Precision: 0.8452860596293312
Accuracy: 0.7988899167437558
F1 Score Weighted: 0.7986218676864166
F1 Score Macro: 0.7987874026842092
Recall: 0.7490182077829347


## TF-IDF With Lemmatization

### Create train and test set

In [67]:
X_lem = pickle.load(open('../pickle/X_lem.pkl', 'rb'))
y_lem = pd.read_pickle('../pickle/y_lem.pkl')

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X_lem, y_lem, test_size=0.20, random_state=15)

In [69]:
# Instantiate TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words= stop_words, ngram_range=(1,2))

# Save Train and Test data
data_train_lem = tfidf.fit_transform(X_train)
data_test_lem = tfidf.transform(X_test)

### Logistic Regression

In [71]:
# Baseline Regression Model
lg_base_tfidf = LogisticRegression(random_state=1)
lg_base_tfidf.fit(data_train_lem, y_train) 
y_lg_base_tfidf_lem = lg_base_tfidf.predict(data_test_lem)
# 28 seconds

In [72]:
# Logistic Regression TF-IDF baseline evaluation
evaluation(y_test, y_lg_base_tfidf_lem)

Evaluation Metrics:
Precision: 0.8558648111332008
Accuracy: 0.8270120259019427
F1 Score Weighted: 0.8268066579512767
F1 Score Macro: 0.8268506653692765
Recall: 0.789908256880734


### Random Forest

In [73]:
rf_base_tfidf = RandomForestClassifier(random_state=1)
rf_base_tfidf.fit(data_train_lem, y_train)
y_rf_base_tfidf = rf_base_tfidf.predict(data_test_lem)

In [74]:
# Logistic Regression TF-IDF baseline evaluation
evaluation(y_test, y_rf_base_tfidf)

Evaluation Metrics:
Precision: 0.8402540302882266
Accuracy: 0.8179925994449584
F1 Score Weighted: 0.8178669502647007
F1 Score Macro: 0.8179009605691314
Recall: 0.7889908256880734


## SMOTE

In [79]:
smote = SMOTE()

In [80]:
X_train_resampled, y_train_resampled = smote.fit_sample(data_train_lem, y_train) 

In [81]:
rf_base_tfidf = RandomForestClassifier(random_state=1)
rf_base_tfidf.fit(X_train_resampled, y_train_resampled)
y_rf_base_tfidf = rf_base_tfidf.predict(data_test_lem)

In [82]:
# Logistic Regression TF-IDF baseline evaluation
evaluation(y_test, y_rf_base_tfidf)

Evaluation Metrics:
Precision: 0.8301610541727672
Accuracy: 0.8087419056429233
F1 Score Weighted: 0.8086145397811295
F1 Score Macro: 0.8086495410866605
Recall: 0.7802752293577981


## Upsampling

## Downsampling

### GridSearch LogReg

In [ ]:
#This is the parameters for the baseline Gridsearch with Logistic Regression
param_dict={
    "class_weight":["balanced"],
    "solver":["lbfgs","liblinear"]
}
logreg = LogisticRegression(max_iter=1000)

In [ ]:
grid_lg = GridSearchCV(logreg, param_dict, cv=20, scoring='f1_weighted', verbose=1, n_jobs=-1)
grid_lg.fit(X_train_cv,y_train_cv)

In [ ]:
y_gc_base_cv = grid_lg(X_test)

In [ ]:
# Logistic Regression baseline evaluation
evaluation(y_test_cv, y_gc_base_cv)